In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('country_charts.csv')
df.head()

,Pos,artist_name,song_title,country
0,1,¥$,"CARNIVAL (w/ Kanye West, Ty Dolla $ign)",United States
1,2,Noah Kahan,Stick Season,United States
2,3,Benson Boone,Beautiful Things,United States
3,4,21 Savage,redrum,United States
4,5,Zach Bryan,I Remember Everything (w/ Kacey Musgraves),United States


# THINGS TO DO
SQL - Multi Index
Look up recommendation systems online
Find 

In [11]:
class DatabaseHandler:
    @staticmethod
    def create_database():
        conn = sqlite3.connect('country_data.db') #creates and connects to country_data.db database
        conn.close()
    

    @staticmethod
    def create_table():
        conn = sqlite3.connect('country_data.db')
        c = conn.cursor()
        #REDO!!!
        #create a table called 'countries' that houses the top songs, its artists, and rankings
        c.execute('''CREATE TABLE IF NOT EXISTS songs (
                    country TEXT,
                    artist_name TEXT,
                    song_name TEXT,
                    rank INTEGER,
                    PRIMARY KEY (country, rank)
                    )''')
        conn.commit()
        conn.close()
# MULTI INDEX SQL
    @staticmethod
    def insert_data(country, artist_name, song_name, rank):
        conn = sqlite3.connect('country_data.db')
        c = conn.cursor()
        #insert country, artist, and song info into the database
        c.execute("INSERT INTO songs (country, rank, song_name, artist_name) VALUES (?, ?, ?, ?)", (country, artist_name, song_name, rank))
        conn.commit()
        conn.close()


    def populate_db(df = pd.read_csv('country_charts.csv')):
        #These two functions below are called to create the database and table in order to populate that db and table
        DatabaseHandler.create_database()  
        DatabaseHandler.create_table()

        #iterate through pd df and call insert_data function to populate the database
        for index, row in df.iterrows():
            DatabaseHandler.insert_data(row['country'], row['artist_name'], row['song_title'], row['Pos'])


    #function to find songs that are unique to that country
    def find_unique_songs(country):
        conn = sqlite3.connect('country_data.db')
        c = conn.cursor()

        #SQL query that finds distinct song_name from the inputted country
        c.execute("""
        SELECT DISTINCT song_name 
        FROM songs 
        WHERE country = ? 
        AND song_name NOT IN (
            SELECT song_name FROM songs WHERE country != ?
        )
    """, (country, country))
        
        unique_songs = c.fetchall()
        conn.close()
        return unique_songs


In [12]:
DatabaseHandler.populate_db()

In [4]:
@staticmethod
#returns a dictionary of each country's unique songs
def get_all_country_unique_songs():
    conn = sqlite3.connect('country_data.db')
    c = conn.cursor()
    
    # getting list of all countries in top charts list
    c.execute("SELECT DISTINCT country FROM songs")
    countries = c.fetchall()
    
    # initialize an empty dictionary
    top_songs_by_country = {}

    for country in countries:
        country_name = country[0]
        
        #MUST DEBUG -> Dictionary duplicates the song 5 times in the dictionary for some reason
    
        # Retrieve the top songs for the current country
        c.execute("""
            SELECT song_name 
            FROM songs 
            WHERE country = ?
            AND song_name NOT IN (
                SELECT song_name 
                FROM songs 
                WHERE country != ?
            ) 
            ORDER BY rank ASC
        """, (country_name, country_name))
        top_songs = c.fetchall()
        
        # Store the top songs in the dictionary
        top_songs_by_country[country_name] = [song[0] for song in top_songs]

    conn.close()
    return top_songs_by_country



In [17]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def collect_user_listening_data():
    # Initialize Spotipy with your client ID, client secret, and redirect URI
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='your_client_id',
                                                   client_secret='your_client_secret',
                                                   redirect_uri='your_redirect_uri',
                                                   scope='user-library-read,user-top-read'))

    # Fetch the user's top artists and tracks
    top_artists = sp.current_user_top_artists(limit=15)
    top_tracks = sp.current_user_top_tracks(limit=200)

    # Process the fetched data as needed
    user_listening_data = {
        'top_artists': [artist['name'] for artist in top_artists['items']],
        'top_tracks': [track['name'] for track in top_tracks['items']]
    }

    return user_listening_data

def recommend_countries(user_listening_data, top_n=3):
    # Load the dataset of top songs in different countries
    top_songs_data = pd.read_csv('top_songs_by_country.csv')  # Example: Replace with your dataset
    
    # Calculate similarity score for each country
    similarity_scores = {}
    for country in top_songs_data['Country']:
        country_top_songs = top_songs_data[top_songs_data['Country'] == country]['SongTitle']
        common_songs = set(user_listening_data['top_tracks']).intersection(country_top_songs)
        similarity_scores[country] = len(common_songs)
    
    # Sort countries based on similarity scores
    sorted_countries = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Extract top N countries
    top_countries = [country for country, score in sorted_countries[:top_n]]
    
    return top_countries



